# Import Needed Libraries

In [1]:
from statistics import mean,stdev
from termcolor import colored
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [2]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)    
    return first_nodes

In [3]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)   
    return bottom_nodes

In [4]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

# LT Simulations for all Networks

In [5]:
networks = next(walk(RealNetworks), (None, None, []))[2]

for network in networks:
    counter = 0
    g = nx.read_gml(f'{RealNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]
    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))
    
    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraphs(max)
    #print(network)
    
    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])
        
    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/RealNetworks/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here
        #print(dictionary)
    
    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_average_each_simulation'] = dict()
        globals()[f'{centrality}_standard_each_simulation'] = dict()
        for fraction in fractions: 
            globals()[f'{centrality}_average_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standard_each_simulation'][fraction] = 0
    
    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')
    
    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')
    
    #thresholds
    randomThresholds = np.random.uniform(low=0, high=1, size=len(g.nodes()))
    fixedThresholds = [0.4,0.7]
    
    #Run diffusion model on centralities
    for counter in range(3): 
        for centrality in colors.keys(): 
            print(colored(f'Counter: {counter}, Centrality: {centrality}', 'green',attrs=['bold']))
            for fraction in fractions:

                globals()[f'l_{centrality}'] = list()
                fraction_infected = round(fraction*network_size)

                if(centrality=="modv_neg"):
                    infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
                else:
                    infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)

                
                
                if(counter==0 or counter==1):
                    model = ep.ThresholdModel(g)
                    config = mc.Configuration()
                    config.add_model_initial_configuration('Infected', infected_nodes)
                    threshold = fixedThresholds[counter]

                    for i in g.nodes():
                        config.add_node_configuration("threshold", i, threshold)

                    model.set_initial_status(config)
              
                    x = 1
                    iterations = model.iteration_bunch(1)
                    iterations = iterations + model.iteration_bunch(1)
                    while(iterations[x-1]['node_count']!=iterations[x]['node_count']):
                        iterations = iterations + model.iteration_bunch(1)
                        x = x+1

                    finalInfectedInOneIteration = iterations[x]['node_count'][1]
                
                    globals()[f'l_{centrality}'].append(finalInfectedInOneIteration)
                    globals()[f'{centrality}_average_each_simulation'][fraction] = finalInfectedInOneIteration
                    #print( globals()[f'{centrality}_average_each_simulation'])
                    p = f"./Pickle Variables/{n}/Threshold-{threshold}/"
                    if(path.exists(p)==False):
                        os.mkdir(p)
                    save_obj(globals()[f'{centrality}_average_each_simulation'], p ,f'{centrality}_average_each_simulation')
                    print(fraction,":",finalInfectedInOneIteration)

                else:
                    for i in range(repeatedSimulations):
                        model = ep.ThresholdModel(g)
                        config = mc.Configuration()
                        config.add_model_initial_configuration('Infected', infected_nodes)
                        random.shuffle(randomThresholds)
                        for i,threshold in zip(g.nodes(),randomThresholds):
                            config.add_node_configuration("threshold", i, threshold)

                        model.set_initial_status(config)
                        x = 1
                        iterations = model.iteration_bunch(1)
                        iterations = iterations + model.iteration_bunch(1)
                        while(iterations[x-1]['node_count']!=iterations[x]['node_count']):
                            iterations = iterations + model.iteration_bunch(1)
                            x = x+1

                        finalInfectedInOneIteration = iterations[x]['node_count'][1]
                        
                        globals()[f'l_{centrality}'].append(finalInfectedInOneIteration)

                    globals()[f'{centrality}_average_each_simulation'][fraction] = mean(globals()[f'l_{centrality}'])
                    globals()[f'{centrality}_standard_each_simulation'][fraction] = stdev(globals()[f'l_{centrality}'])
                    print(fraction,":", mean(globals()[f'l_{centrality}']))
                    p = f"./Pickle Variables/{n}/UniformRandomThresholds/"
                    if(path.exists(p)==False):
                        os.mkdir(p)
                    save_obj(globals()[f'{centrality}_average_each_simulation'], p ,f'{centrality}_average_each_simulation')
                    save_obj(globals()[f'{centrality}_standard_each_simulation'], p ,f'{centrality}_standard_each_simulation')

Simulation currently on network board_of_directors.gml : Name: 
Type: Graph
Number of nodes: 854
Number of edges: 2745
Average degree:   6.4286...
Counter: 0, Centrality: modv_pos
0.01 : 9
0.02 : 20
0.03 : 30
0.04 : 50
0.05 : 65
0.06 : 74
0.07 : 85
0.08 : 94
0.09 : 116
0.1 : 127
0.11 : 145
0.12 : 157
0.13 : 178
0.14 : 192
0.15 : 209
0.16 : 231
0.17 : 238
0.18 : 250
0.19 : 265
0.2 : 271
0.21 : 298
0.22 : 310
0.23 : 316
0.24 : 327
0.25 : 347
0.26 : 358
0.27 : 364
0.28 : 377
0.29 : 431
0.3 : 436
0.31 : 450
0.32 : 463
0.33 : 472
0.34 : 482
0.35 : 487
0.36 : 498
0.37 : 524
0.38 : 533
0.39 : 567
0.4 : 577
0.41 : 604
0.42 : 615
0.43 : 629
0.44 : 646
0.45 : 654
0.46 : 678
0.47 : 678
0.48 : 682
0.49 : 690
0.5 : 694
Counter: 0, Centrality: com
0.01 : 17
0.02 : 37
0.03 : 66
0.04 : 74
0.05 : 90
0.06 : 95
0.07 : 115
0.08 : 129
0.09 : 138
0.1 : 158
0.11 : 183
0.12 : 193
0.13 : 229
0.14 : 264
0.15 : 294
0.16 : 330
0.17 : 334
0.18 : 370
0.19 : 377
0.2 : 377
0.21 : 377
0.22 : 383
0.23 : 395
0.24 : 395


0.17 : 150
0.18 : 162
0.19 : 167
0.2 : 174
0.21 : 182
0.22 : 193
0.23 : 199
0.24 : 208
0.25 : 217
0.26 : 225
0.27 : 234
0.28 : 242
0.29 : 252
0.3 : 259
0.31 : 270
0.32 : 279
0.33 : 288
0.34 : 295
0.35 : 304
0.36 : 311
0.37 : 322
0.38 : 334
0.39 : 342
0.4 : 350
0.41 : 357
0.42 : 366
0.43 : 374
0.44 : 385
0.45 : 397
0.46 : 406
0.47 : 415
0.48 : 421
0.49 : 426
0.5 : 433
Counter: 1, Centrality: pc
0.01 : 9
0.02 : 17
0.03 : 28
0.04 : 36
0.05 : 45
0.06 : 52
0.07 : 60
0.08 : 68
0.09 : 77
0.1 : 85
0.11 : 94
0.12 : 102
0.13 : 111
0.14 : 120
0.15 : 132
0.16 : 140
0.17 : 147
0.18 : 158
0.19 : 166
0.2 : 178
0.21 : 187
0.22 : 196
0.23 : 204
0.24 : 213
0.25 : 220
0.26 : 227
0.27 : 239
0.28 : 245
0.29 : 255
0.3 : 265
0.31 : 276
0.32 : 282
0.33 : 291
0.34 : 299
0.35 : 308
0.36 : 317
0.37 : 321
0.38 : 329
0.39 : 337
0.4 : 348
0.41 : 354
0.42 : 364
0.43 : 373
0.44 : 379
0.45 : 389
0.46 : 397
0.47 : 404
0.48 : 415
0.49 : 421
0.5 : 431
Counter: 1, Centrality: kshell
0.01 : 9
0.02 : 20
0.03 : 26
0.04 : 36


0.08 : 187.97
0.09 : 231.25
0.1 : 252.21
0.11 : 291.75
0.12 : 304.62
0.13 : 314.83
0.14 : 330.92
0.15 : 344.31
0.16 : 371.78
0.17 : 394.6
0.18 : 406.2
0.19 : 411.25
0.2 : 423.62
0.21 : 445
0.22 : 460.74
0.23 : 467.76
0.24 : 473.01
0.25 : 478.22
0.26 : 478.57
0.27 : 500.93
0.28 : 506.57
0.29 : 514.89
0.3 : 520.87
0.31 : 525.71
0.32 : 533.84
0.33 : 535.66
0.34 : 539.16
0.35 : 544.14
0.36 : 557.15
0.37 : 565.36
0.38 : 576.25
0.39 : 584.12
0.4 : 595.17
0.41 : 601.65
0.42 : 606.17
0.43 : 610.25
0.44 : 613.74
0.45 : 614.4
0.46 : 627.42
0.47 : 642.51
0.48 : 651.08
0.49 : 653.59
0.5 : 661.44
Counter: 2, Centrality: cbc
0.01 : 93.87
0.02 : 140.56
0.03 : 195
0.04 : 250.63
0.05 : 274.37
0.06 : 275.05
0.07 : 293.25
0.08 : 330.4
0.09 : 336.53
0.1 : 373.28
0.11 : 384.7
0.12 : 391.67
0.13 : 398
0.14 : 428.92
0.15 : 447.81
0.16 : 480.11
0.17 : 486.18
0.18 : 501.37
0.19 : 513.85
0.2 : 527.73
0.21 : 538.01
0.22 : 546.01
0.23 : 550.79
0.24 : 556.66
0.25 : 557.96
0.26 : 565.7
0.27 : 586.84
0.28 : 586.36
0

0.48 : 105
0.49 : 105
0.5 : 105
Counter: 0, Centrality: modv_abs
0.01 : 1
0.02 : 2
0.03 : 9
0.04 : 10
0.05 : 15
0.06 : 15
0.07 : 16
0.08 : 16
0.09 : 16
0.1 : 17
0.11 : 29
0.12 : 30
0.13 : 65
0.14 : 68
0.15 : 68
0.16 : 70
0.17 : 70
0.18 : 70
0.19 : 70
0.2 : 70
0.21 : 73
0.22 : 73
0.23 : 105
0.24 : 105
0.25 : 105
0.26 : 105
0.27 : 105
0.28 : 105
0.29 : 105
0.3 : 105
0.31 : 105
0.32 : 105
0.33 : 105
0.34 : 105
0.35 : 105
0.36 : 105
0.37 : 105
0.38 : 105
0.39 : 105
0.4 : 105
0.41 : 105
0.42 : 105
0.43 : 105
0.44 : 105
0.45 : 105
0.46 : 105
0.47 : 105
0.48 : 105
0.49 : 105
0.5 : 105
Counter: 0, Centrality: degree
0.01 : 1
0.02 : 5
0.03 : 54
0.04 : 55
0.05 : 67
0.06 : 105
0.07 : 105
0.08 : 105
0.09 : 105
0.1 : 105
0.11 : 105
0.12 : 105
0.13 : 105
0.14 : 105
0.15 : 105
0.16 : 105
0.17 : 105
0.18 : 105
0.19 : 105
0.2 : 105
0.21 : 105
0.22 : 105
0.23 : 105
0.24 : 105
0.25 : 105
0.26 : 105
0.27 : 105
0.28 : 105
0.29 : 105
0.3 : 105
0.31 : 105
0.32 : 105
0.33 : 105
0.34 : 105
0.35 : 105
0.36 : 10

0.46 : 64.54
0.47 : 65.87
0.48 : 65.64
0.49 : 68.15
0.5 : 70.34
Counter: 2, Centrality: com
0.01 : 2.98
0.02 : 9.51
0.03 : 12.94
0.04 : 19.36
0.05 : 22.23
0.06 : 23.88
0.07 : 31.18
0.08 : 36.08
0.09 : 37.56
0.1 : 45.19
0.11 : 50.17
0.12 : 52.53
0.13 : 56.34
0.14 : 58.09
0.15 : 59.1
0.16 : 60.96
0.17 : 60.97
0.18 : 60.54
0.19 : 62.72
0.2 : 64.44
0.21 : 66.58
0.22 : 67.48
0.23 : 69.57
0.24 : 71.09
0.25 : 71.33
0.26 : 67.92
0.27 : 71.77
0.28 : 73.01
0.29 : 73.68
0.3 : 78.64
0.31 : 78.75
0.32 : 80.75
0.33 : 85.16
0.34 : 86.66
0.35 : 87.07
0.36 : 88.32
0.37 : 88.64
0.38 : 89.49
0.39 : 90.11
0.4 : 90.94
0.41 : 89.81
0.42 : 91.07
0.43 : 89.71
0.44 : 89.59
0.45 : 89.57
0.46 : 91.78
0.47 : 93.07
0.48 : 91.54
0.49 : 91.59
0.5 : 93.02
Counter: 2, Centrality: cbm
0.01 : 6.3
0.02 : 13.93
0.03 : 17.49
0.04 : 21.98
0.05 : 26.34
0.06 : 31.19
0.07 : 35.94
0.08 : 37.4
0.09 : 39.91
0.1 : 48.41
0.11 : 50.84
0.12 : 51.69
0.13 : 53.95
0.14 : 54.71
0.15 : 54.81
0.16 : 61.27
0.17 : 64.38
0.18 : 63.99
0.19 : 6

...

0.28 : 162
0.29 : 167
0.3 : 170
0.31 : 172
0.32 : 177
0.33 : 182
0.34 : 192
0.35 : 199
0.36 : 201
0.37 : 207
0.38 : 212
0.39 : 217
0.4 : 222
0.41 : 224
0.42 : 228
0.43 : 230
0.44 : 238
0.45 : 243
0.46 : 245
0.47 : 250
0.48 : 255
0.49 : 260
0.5 : 262
Counter: 1, Centrality: cbm
0.01 : 8
0.02 : 15
0.03 : 19
0.04 : 27
0.05 : 35
0.06 : 42
0.07 : 47
0.08 : 50
0.09 : 62
0.1 : 75
0.11 : 81
0.12 : 83
0.13 : 89
0.14 : 97
0.15 : 102
0.16 : 110
0.17 : 118
0.18 : 128
0.19 : 137
0.2 : 143
0.21 : 150
0.22 : 153
0.23 : 164
0.24 : 168
0.25 : 175
0.26 : 175
0.27 : 175
0.28 : 176
0.29 : 177
0.3 : 180
0.31 : 183
0.32 : 183
0.33 : 183
0.34 : 183
0.35 : 185
0.36 : 187
0.37 : 190
0.38 : 192
0.39 : 192
0.4 : 196
0.41 : 200
0.42 : 202
0.43 : 208
0.44 : 210
0.45 : 216
0.46 : 216
0.47 : 219
0.48 : 226
0.49 : 230
0.5 : 234
Counter: 1, Centrality: chb
0.01 : 8
0.02 : 12
0.03 : 16
0.04 : 19
0.05 : 23
0.06 : 32
0.07 : 32
0.08 : 35
0.09 : 38
0.1 : 48
0.11 : 53
0.12 : 56
0.13 : 60
0.14 : 65
0.15 : 67
0.16 : 71
0.17 :

0.44 : 256.5
0.45 : 253.56
0.46 : 258.87
0.47 : 255.86
0.48 : 259.54
0.49 : 261.05
0.5 : 267.73
Counter: 2, Centrality: pc
0.01 : 31.04
0.02 : 61.16
0.03 : 93.52
0.04 : 102.79
0.05 : 134.73
0.06 : 139.12
0.07 : 144.97
0.08 : 146.52
0.09 : 147.86
0.1 : 156.78
0.11 : 176.84
0.12 : 181.02
0.13 : 187.92
0.14 : 190.46
0.15 : 197.12
0.16 : 211.24
0.17 : 229.83
0.18 : 228.74
0.19 : 228.75
0.2 : 232.77
0.21 : 240.49
0.22 : 246.6
0.23 : 252.61
0.24 : 257.76
0.25 : 262.35
0.26 : 262.86
0.27 : 266.59
0.28 : 272.85
0.29 : 276.17
0.3 : 284.26
0.31 : 283.08
0.32 : 290.08
0.33 : 296.74
0.34 : 303.91
0.35 : 308.02
0.36 : 309.04
0.37 : 306.98
0.38 : 309.46
0.39 : 308.44
0.4 : 309.53
0.41 : 310.33
0.42 : 311.8
0.43 : 310.71
0.44 : 313.08
0.45 : 313.75
0.46 : 315.45
0.47 : 315.29
0.48 : 314.6
0.49 : 315.37
0.5 : 318.53
Counter: 2, Centrality: kshell
0.01 : 44.54
0.02 : 73.17
0.03 : 72.92
0.04 : 95.83
0.05 : 107.02
0.06 : 113
0.07 : 116.31
0.08 : 118.92
0.09 : 140.84
0.1 : 158.67
0.11 : 173.8
0.12 : 178.0

0.31 : 1622
0.32 : 1622
0.33 : 1622
0.34 : 1622
0.35 : 1622
0.36 : 1622
0.37 : 1622
0.38 : 1622
0.39 : 1622
0.4 : 1622
0.41 : 1622
0.42 : 1622
0.43 : 1622
0.44 : 1622
0.45 : 1622
0.46 : 1622
0.47 : 1622
0.48 : 1622
0.49 : 1622
0.5 : 1628
Counter: 0, Centrality: kshell
0.01 : 156
0.02 : 1622
0.03 : 1622
0.04 : 1622
0.05 : 1622
0.06 : 1622
0.07 : 1622
0.08 : 1622
0.09 : 1622
0.1 : 1622
0.11 : 1622
0.12 : 1622
0.13 : 1622
0.14 : 1622
0.15 : 1622
0.16 : 1622
0.17 : 1622
0.18 : 1622
0.19 : 1622
0.2 : 1622
0.21 : 1622
0.22 : 1622
0.23 : 1622
0.24 : 1622
0.25 : 1622
0.26 : 1622
0.27 : 1623
0.28 : 1623
0.29 : 1623
0.3 : 1623
0.31 : 1623
0.32 : 1623
0.33 : 1623
0.34 : 1623
0.35 : 1629
0.36 : 1629
0.37 : 1629
0.38 : 1641
0.39 : 1641
0.4 : 1641
0.41 : 1641
0.42 : 1641
0.43 : 1641
0.44 : 1641
0.45 : 1646
0.46 : 1646
0.47 : 1646
0.48 : 1646
0.49 : 1652
0.5 : 1652
Counter: 0, Centrality: cbc
0.01 : 289
0.02 : 1622
0.03 : 1622
0.04 : 1622
0.05 : 1622
0.06 : 1622
0.07 : 1622
0.08 : 1622
0.09 : 1622
0.

0.5 : 1203
Counter: 1, Centrality: modv_neg
0.01 : 27
0.02 : 58
0.03 : 91
0.04 : 125
0.05 : 162
0.06 : 199
0.07 : 246
0.08 : 276
0.09 : 321
0.1 : 365
0.11 : 430
0.12 : 473
0.13 : 494
0.14 : 521
0.15 : 550
0.16 : 557
0.17 : 574
0.18 : 589
0.19 : 603
0.2 : 649
0.21 : 673
0.22 : 708
0.23 : 727
0.24 : 747
0.25 : 752
0.26 : 791
0.27 : 810
0.28 : 816
0.29 : 826
0.3 : 840
0.31 : 850
0.32 : 859
0.33 : 865
0.34 : 877
0.35 : 892
0.36 : 904
0.37 : 919
0.38 : 935
0.39 : 942
0.4 : 983
0.41 : 994
0.42 : 1005
0.43 : 1018
0.44 : 1031
0.45 : 1044
0.46 : 1057
0.47 : 1069
0.48 : 1078
0.49 : 1089
0.5 : 1112
Counter: 1, Centrality: modv_abs
0.01 : 27
0.02 : 65
0.03 : 99
0.04 : 130
0.05 : 161
0.06 : 199
0.07 : 235
0.08 : 271
0.09 : 289
0.1 : 329
0.11 : 360
0.12 : 382
0.13 : 399
0.14 : 442
0.15 : 471
0.16 : 488
0.17 : 551
0.18 : 586
0.19 : 615
0.2 : 679
0.21 : 706
0.22 : 733
0.23 : 782
0.24 : 851
0.25 : 931
0.26 : 969
0.27 : 1002
0.28 : 1020
0.29 : 1041
0.3 : 1057
0.31 : 1072
0.32 : 1129
0.33 : 1155
0.34 : 1

0.19 : 1299.01
0.2 : 1311.34
0.21 : 1306.59
0.22 : 1318.93
0.23 : 1320.49
0.24 : 1328.39
0.25 : 1318.12
0.26 : 1343.44
0.27 : 1343.21
0.28 : 1353.99
0.29 : 1360.55
0.3 : 1367.81
0.31 : 1373.03
0.32 : 1370.93
0.33 : 1378.4
0.34 : 1380.54
0.35 : 1388.26
0.36 : 1395.74
0.37 : 1396.67
0.38 : 1411.6
0.39 : 1410.22
0.4 : 1411.8
0.41 : 1423.06
0.42 : 1417.82
0.43 : 1419.11
0.44 : 1428.37
0.45 : 1435.43
0.46 : 1434.28
0.47 : 1442.82
0.48 : 1447.55
0.49 : 1457.94
0.5 : 1458.51
Counter: 2, Centrality: modv_abs
0.01 : 773.84
0.02 : 946.32
0.03 : 1035.98
0.04 : 1097.9
0.05 : 1147.21
0.06 : 1181.79
0.07 : 1224.57
0.08 : 1238.62
0.09 : 1269.54
0.1 : 1289.81
0.11 : 1314.18
0.12 : 1340.84
0.13 : 1351.77
0.14 : 1359.49
0.15 : 1373.28
0.16 : 1387.86
0.17 : 1406.12
0.18 : 1404.94
0.19 : 1420.95
0.2 : 1430.37
0.21 : 1440.87
0.22 : 1463.53
0.23 : 1469.81
0.24 : 1474.48
0.25 : 1484.82
0.26 : 1496.17
0.27 : 1507.98
0.28 : 1509.96
0.29 : 1522.65
0.3 : 1523.13
0.31 : 1530.84
0.32 : 1535.72
0.33 : 1540.64
0.34 